In [1]:
import uuid
import random
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from create_sap_table import create_table_leanx as ctl

import values, helpers
import master_data, text_data, sales_doc_data

In [2]:
# master tables
for method in (
    master_data.users,
    master_data.customers_and_vendors, 
    master_data.plants, 
    master_data.materials, 
    master_data.material_support,
    master_data.routes,
    master_data.company_codes
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        df = pd.concat([all_cols, pd.DataFrame(v.values())])
        df.to_csv(f'data/OCPM/master/{table_name}.csv', index=False)

In [3]:
# text tables
for method in (
    text_data.domain_fixed_values, 
    text_data.doc_types, 
    text_data.organization,
    text_data.distribution,
    text_data.sales_doc_item_categories,
    text_data.sales_doc_rejection_reasons,
    text_data.system_status,
    text_data.blocking_reasons
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        df = pd.concat([all_cols, pd.DataFrame(v.values())])
        df.to_csv(f'data/OCPM/text/{table_name}.csv', index=False)

In [2]:
def get_user_name(automation_probability: float):
    if random.random() <= automation_probability:
        return 'BATCH_JOB'
    return random.choice(list(values.om_users.keys()))

In [5]:
# sales order tables
sales_doc_tables = {
    'VBAK_json': {},
    'VBKD_json': {},
    'VBUK_json': {},
    'VBAP_json': {},
    'VBEP_json': {},
    'LIKP_json': {},
    'LIPS_json': {},
    'EKBE_json': {}, 
    'MSEG_json': {},
    'VBRK_json': {},
    'VBRP_json': {},
    'BKPF_json': {},
    'BSEG_json': {},
    'CDHDR_json': {},
    'CDPOS_json': {},
    'JCDS_json': {},
    'VBFA_json': {}
}

for i in range(52_319):
    transition_prob = np.array([
        [0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], # 0: Approve Sales Order
        [0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], # 1: Generate Delivery Document
        [0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], # 2: Pick Items
        [0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], # 3: Post Goods Issue
        [0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], # 4: Send Invoice
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], # 5: Receive Delivery Confirmation
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # 6: Clear Invoice

        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # 7: Reject Sales Order
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.85, 0.00, 0.00, 0.15], # 8: Set Credit Block
        [0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], # 9: Release Credit Block
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.85, 0.15], # 10: Set Delivery Block
        [0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], # 11: Release Delivery Block
        
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00] # 12: Terminate
    ])

    """
    Steps map
    Starts at:  Create Sales Order
    0: Approve Sales Order
    1: Generate Delivery Document
    2: Pick Items
    3: Post Goods Issue
    4: Send Invoice
    5: Recieve Delivery Confirmation
    6: Clear Invoice

    7: Reject Sales Order
    8: Set Credit Block
    9: Release Credit Block
    10: Set Delivery Block
    11: Release Delivery Block

    12: Terminate Process
    """

    num_items = random.randint(1, 25)
    materials = random.sample(list(values.om_materials.keys()), num_items)
    quantities = [random.randint(12, 36)*12 for _ in range(num_items)] # x dozens of everything
    latest_date = helpers.generate_random_date(start_date=datetime(2021, 1, 1), end_date=datetime(2023, 9, 1))
    latest_time = helpers.generate_random_time()
    sales_org=random.choice(list(values.sales_orgs.keys()))
    so_created_by = get_user_name(0.3)
    customer=values.om_customers[random.choice(list(values.om_customers.keys()))]
    reqested_delivery_date=helpers.add_random_days(min_days=3, max_days=10, current_date=latest_date)

    sd = sales_doc_data.SalesAndDistribution(
        vbeln=f'{str(uuid.uuid4())[-17:]}', 
        materials=materials, 
        quantities=quantities, 
        customer=customer,
        start_date=latest_date
    )

    sd.create_sales_order(
        sales_org=sales_org,
        reqested_delivery_date=reqested_delivery_date,
        shipping_condition=random.choice(list(values.shipping_conditions.keys())),
        sales_office=random.choice(list(values.sales_orgs[sales_org]['sales_offices'].keys())),
        distribution_channel=random.choice(list(values.sales_orgs[sales_org]['distribution_channels'].keys())),
        erdat=latest_date,
        ernam=so_created_by,
        atime=latest_time
    )
    # latest_date += helpers.UPTO_WEEK()
    latest_time = helpers.add_random_hours(1, latest_time)

    # decide on apprve or reject sales order
    def approve_or_reject_so():
        if so_created_by != 'BATCH_JOB':
            if random.random() < 0.2:
                return 7 # Reject Sales Oreder for inaccurate information
        return 0 # Approve
    
    step = approve_or_reject_so()
    while step != 12: # until termination
        if step == 0:
            sd.approve_sales_order(
                usnam=get_user_name(0.6),
                udate=latest_date,
                atime=latest_time
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            
            # Set Credit Block?
            if customer['credit_risk'] > random.random():
                transition_prob[step][8] = 0.5
                latest_date = helpers.add_random_days(1, 3, latest_date)
            else:
                latest_date = helpers.add_random_days(1, 1, latest_date)

        elif step == 1:
            sd.generate_delivery_document(
                ernam=get_user_name(0.3), 
                erdat=latest_date,
                planned_delivery_date=latest_date + helpers.UPTO_WEEK(),
                picking_date=None,
                delivery_date=None,
                confirmation_date=None,
                atime=latest_time
            )
            latest_time = helpers.add_random_hours(3, latest_time)

            # Set Delivery Block?
            avg_availability = np.average(np.array([values.om_materials[m]['availability'] for m in materials]))
            if avg_availability < 0.5:
                latest_date = helpers.add_random_days(1, 3, latest_date)
                transition_prob[step][10] = 0.5
            else:
                latest_date = helpers.add_random_days(1, 1, latest_date)
        elif step == 2:
            sd.pick_items(
                usnam=get_user_name(0.2), 
                udate=latest_date,
                atime=helpers.generate_random_time()
            )
            latest_time = helpers.add_random_hours(8, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 3:
            sd.post_goods_issue(
                cpudt=latest_date,
                usnam=get_user_name(0.3),
                atime=helpers.generate_random_time()
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, reqested_delivery_date.day, latest_date)
        elif step == 4:
            sd.create_invoice(
                ernam=get_user_name(0.5),
                erdat=latest_date,
                atime=helpers.generate_random_time()
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 7, latest_date)
        elif step == 5:
            sd.delivery_confirmation(
                usnam=get_user_name(0.6), 
                udate=latest_date
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 14, latest_date)
        elif step == 6:
            sd.clear_debit_invoice(
                cpudt=latest_date,
                usnam=get_user_name(0.6),
                cleared_date=latest_date + helpers.UPTO_WEEK(),
                atime=helpers.generate_random_time()
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        
        elif step == 7:
            sd.reject_sales_order(
                udate=latest_date,
                usnam=get_user_name(0.2), 
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 8:
            sd.set_credit_block(
                udate=latest_date,
                usnam=get_user_name(0.2)
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 3, latest_date)
        elif step == 9:
            sd.release_credit_block(
                udate=latest_date,
                usnam=get_user_name(0.1), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 10:
            sd.set_delivery_block(
                udate=latest_date,
                usnam=get_user_name(0.3), 
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 3, latest_date)
        elif step == 11:
            sd.release_delivery_block(
                udate=latest_date,
                usnam=get_user_name(0.1), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)

         # update transition matrix to sum to 1
        transition_prob = transition_prob / transition_prob.sum(axis=1, keepdims=True)
        step = np.random.choice(13, p=transition_prob[step])

    for k, v in sd.tables.items():
        for entry_key in list(v.keys()):
            sales_doc_tables[k][entry_key] = sd.tables[k][entry_key]

In [6]:
for table, rows in sales_doc_tables.items():
    table_name = table.split('_')[0]
    all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
    df = pd.concat([all_cols, pd.DataFrame(rows.values())])
    df.to_csv(f'data/OCPM/sales-document/{table_name}.csv', index=False)